# Tier 3, Week 6: Variational Algorithms & QAOA

**Audience:** Undergraduate / Advanced

**Objective:** Introduce the hybrid quantum-classical model of computation. Formalize the structure of Variational Algorithms, specifically the QAOA ansatz. Connect this to the Qiskit Primitives (Estimator) used to run them.

## 1. Bridging from Week 5: Why Do We Need a New Method?

In Week 5, we learned about the devastating effect of **noise** and **decoherence**.

Our formal `Statevector` math from Weeks 1-4 only works for ideal, perfect systems. Real "NISQ" (Noisy Intermediate-Scale Quantum) devices lose their "quantum-ness" (coherence) very quickly.

**Thus,** we *cannot* run long, complex algorithms (like Shor's algorithm) that require millions of gates at the current time where we do not have great breakthrough that can sustain the function of millions of gates. The noise will destroy the answer.

**The Solution: The Hybrid Quantum-Classical Loop**, instead of one long quantum algorithm, we use a combination of *2 types of computer*:

1. A **Quantum Computer** runs a very *shallow* (short) circuit with tunable parameters. Its only job is to prepare a "trial state" and measure a single value (the "cost").

2. A **Classical Computer** does all the "thinking." It takes the cost, updates the parameters, and tells the quantum computer to try again, repeating the process in a loop.

This is the core idea of all **Variational Algorithms**.

## 2. The Variational Algorithm Model

Variational algorithms are a broad class of hybrid algorithms. The **Variational Quantum Eigensolver (VQE)** is the most general version, designed to find the ground state energy of *any* Hamiltonian (often used in quantum chemistry). QAOA, which we are focusing on, is a specific *type* of VQE tailored for classical optimization problems.

This hybrid loop is an optimization process, which maps directly to the Tier 1,2 concepts of "finding the lowest point in a valley."

The loop works like this:

1. **Classical Optimizer:** Picks an initial set of parameters (angles), $\vec{\theta}$.

2. **Quantum Computer (QPU):**

   * Prepares a simple state (e.g., $|+\rangle^{\otimes n}$).

   * Runs the "Ansatz" circuit $U(\vec{\theta})$ (our shallow, parameterized circuit).

   * This prepares the trial state: $|\psi(\vec{\theta})\rangle = U(\vec{\theta})|+\rangle^{\otimes n}$.

3. **Qiskit Estimator (Primitive):**

   * Measures the **Expectation Value** (or "Cost") of this state with respect to a problem Hamiltonian, $H_P$.

   * Calculates: $C = \langle\psi(\vec{\theta})| H_P |\psi(\vec{\theta})\rangle$.

4. **Classical Optimizer:**

   * Receives the single number, $C$.

   * Checks if $C$ is the minimum. If not, it uses a classical algorithm (e.g., COBYLA, SPSA, or gradient descent) to pick a *new* set of parameters $\vec{\theta}'$.

   * The loop repeats until the cost $C$ is minimized.

### The Classical Optimizer in Practice

While gradient descent is a valid optimizer, calculating gradients on a quantum computer (e.g., using the "parameter shift rule") can be very slow and requires many extra circuit executions.

In practice, **gradient-free** optimizers are very common in Qiskit because they are more robust to noise:

* **COBYLA:** A widely used, reliable optimizer.

* **SPSA:** A powerful optimizer for *real, noisy hardware*. It cleverly estimates the gradient for all $2p$ parameters using only two measurements, making it very efficient.

The quantum computer's only job is to provide the cost value $C$. The classical computer does all the optimization.

## 3. QAOA Example: Max-Cut & The Cost Hamiltonian

Variational algorithms are built to solve optimization problems. The classic example is **Max-Cut**. First, we have to define a Cost Hamiltonian to solve the Max Cut problem. Before we delve into Cost Hamiltonian, we discuss the following first:

**What is "Cost"?** (A Computer Science Idea): In an optimization problem like Max-Cut, "cost" is just a score. In this section, we design a low score (cost) is good (many cut edges), and a high score is bad (many uncut edges). For the bitstring $01$, the cost is low. For the bitstring $00$, the cost is high.

**What is "Energy"?** (A Physics Idea): In quantum mechanics, "energy" is a physical property of a state, determined by its Hamiltonian $(H)$. A fundamental rule of physics is that systems always try to settle into their lowest possible energy state, called the "ground state."

### Cost Hamiltonian

The **Cost Hamiltonian** (or Problem Hamiltonian, $H_P$) is a quantum operator we *design* to translate a classical optimization problem into a physics problem.

It is constructed so that its **lowest energy state (its "ground state") corresponds to the best possible solution** to the classical problem.

* Finding the **best solution** (e.g., the Max-Cut) becomes the same as...

* ...finding the **ground state** of the Hamiltonian $H_P$.

The **"cost"** of any trial solution (a bitstring) is simply the energy (the expectation value) of that state: $C = \langle\psi| H_P |\psi\rangle$. A lower energy means a better solution.

### Cost Hamiltonian (The Method for Max-Cut)

**Step 1. Map the problem to qubits:**

* **Problem:** Divide nodes into two sets (0 or 1) to *maximize* cut edges.

* **Mapping:** One node = One qubit.

  * Measuring $|0\rangle \implies$ Node is in set 0.

  * Measuring $|1\rangle \implies$ Node is in set 1.

**Step 2. Assign energy penalties/rewards:**

* Our goal is to *maximize* cut edges. This is the same as *minimizing* uncut edges.

* An **uncut edge** (qubits are $|00\rangle$ or $|11\rangle$) is "bad." We give it a *high* energy penalty: **+1**.

* A **cut edge** (qubits are $|01\rangle$ or $|10\rangle$) is "good." We give it a *low* energy reward: **-1**.

**Step 3. Find the quantum operator:**

* We need an operator that gives `+1` for $|00\rangle$/$|11\rangle$ and `-1` for $|01\rangle$/$|10\rangle$. The $Z_i Z_j$ operator is perfect for this:

  * $Z_i Z_j |00\rangle = (+1)(+1)|00\rangle = \mathbf{+1} |00\rangle$  *(High cost)*

  * $Z_i Z_j |11\rangle = (-1)(-1)|11\rangle = \mathbf{+1} |11\rangle$  *(High cost)*

  * $Z_i Z_j |01\rangle = (+1)(-1)|01\rangle = \mathbf{-1} |01\rangle$  *(Low cost)*

  * $Z_i Z_j |10\rangle = (-1)(+1)|10\rangle = \mathbf{-1} |10\rangle$  *(Low cost)*

**Step 4. Sum all costs:**

* For this problem, we do not consider about the weightage of each nodes. Then, the total cost for the whole graph is the sum of the costs for each individual edge. This gives the final formula:

  $$ 
  H_P = \sum_{(i, j) \in \text{edges}} Z_i Z_j
  $$

* Therefore, **minimizing** the energy $\langle H_P \rangle$ is mathematically identical to **maximizing the number of cuts**.

The derivation and steps above is introduced as simple as possible. The full derivation is actually start by **defining a function problem $f(x)$**, then mapping to the Quadratic Unconstrained Binary Optimization **(QUBO)** problems notation, then mapping to Ising Hamiltonian form. For more information, can refer to :
https://quantum.cloud.ibm.com/docs/en/tutorials/quantum-approximate-optimization-algorithm

## 4. The Formal QAOA Ansatz

The "Ansatz" is the formal name for the shallow, parameterized circuit ($U(\vec{\theta})$) that the quantum computer runs. For QAOA, the parameters are $\vec{\beta}$ and $\vec{\gamma}$.

Also, note that for a quantum system with a Hamiltonian $H$ that evolves for a time $t$ undergoes a unitary transformation $U = e^{-iHt}$ (assuming $\hbar=1$).

The ansatz is built in $p$ layers (where $p$ is a small integer, e.g., $p=1, 2, 3...$):

1. **Start State:** We begin in an equal superposition of all possible answers: $|+\rangle^{\otimes n}$.

2. **Layer 1 (Cost):** Apply the **Problem Hamiltonian** $H_P$ for a time $\gamma_1$.

   * $U_P(\gamma_1) = e^{-i\gamma_1 H_P} = \prod_{(i,j)} e^{-i\gamma_1 Z_i Z_j}$

   * This is a circuit of $R_{zz}(2\gamma_1)$ gates on all connected qubits.

3. **Layer 1 (Mixer):** Apply a "Mixer Hamiltonian" $H_M = \sum_i X_i$ for a time $\beta_1$.

   Note: Mixer Hamiltonian in Max Cut is defined as: $H_M = \sum_i X_i$ , where it function as giving the quantum state the ability to **"mix"** between solutions. It can hop from the state $|011\rangle$ to $|111\rangle$ or $|001\rangle$, etc., creating a quantum interference so that all the "bad" solutions destructively interfere (cancel out) and the "good" solutions constructively interfere (add up) as a result. We will discuss it later.

   * $U_M(\beta_1) = e^{-i\beta_1 H_M} = \prod_i e^{-i\beta_1 X_i}$

   * This is a circuit of $R_x(2\beta_1)$ gates on *all* qubits.

This $U_P, U_M$ block is one "layer." The full ansatz repeats this $p$ times:

$$ 
|\psi(\vec{\gamma}, \vec{\beta})\rangle = U_M(\beta_p) U_P(\gamma_p) \dots U_M(\beta_1) U_P(\gamma_1) |+\rangle^{\otimes n}
$$ 

**The Role of the Mixer Hamiltonian**

The choice of $H_M = \sum_i X_i$ is deliberate. Its ground state is $|+\rangle^{\otimes n}$, which is an equal superposition of all $2^n$ possible answer states. Applying $U_M(\beta)$ (which are $R_x$ gates) "mixes" the states, allowing the algorithm to explore the entire solution space and preventing it from getting "stuck" in a local minimum. For problems with *constraints* (e.g., "find the best solution that has *exactly* 3 nodes in set '1'"), this mixer is inefficient because it explores invalid states. In those cases, different "constraint-preserving" mixers are used, such as an **XY-Mixer**.

The number of layers, $p$, is the most important hyperparameter. It controls the fundamental trade-off of the algorithm:  **As** $p \to \infty$**,** The QAOA ansatz is guaranteed by the adiabatic theorem to produce the *exact* optimal solution.

For small $p$, the ansatz is only an *approximation*, as $p$ increases(More layers), the approximation quality gets better. However, deeper circuit is more vulnerable to **decoherence**. Also, the classical optimizer has to find the best values for $2p$ parameters. As $p$ increases, the "cost landscape" can become flat, creating "***barren plateaus***" where the gradient is zero, making it very hard for the optimizer to find the right direction. The 2p parameters $(\gamma_1, \beta_1, \dots, \gamma_p, \beta_p)$ are the $\vec{\theta}$ that the classical optimizer will tune.

**Note:** “Cost landscapes” is a way to visualize the optimization problems. It is plot with **cost function** vs. **parameters $\theta$,** where we will see the  coordinates at the lowest point of the graph is the best parameters to give lowest cost, which is the solution that we need for optimization problem. **Barren plateaus,** as mentioned above is the case where the whole graph appears almost perfectly flat. This is the problem arise where we don’t have lowest point of coordinates on a almost flat graph.

## 5. The Qiskit Primitive: The "Estimator"

Before we can run the hybrid loop on a real device, the abstract circuit must be adapted to the physical constraints of the hardware.

### **Circuit Optimization (Transpilation)**

Imagine you write a program in Python. Before your computer's processor can run it, that high-level code needs to be translated into machine code (0s and 1s) that the specific hardware understands. In Qiskit, **Transpilation** is that translation and optimization process.

You design your QAOA circuit using "virtual" or abstract qubits (e.g., qubit 0, qubit 1, qubit 2) and ideal gates. However, real quantum hardware (like an IBM Quantum Eagle processor) has physical constraints:

**Limited Connectivity:** Not every qubit is connected to every other qubit. You can only perform 2-qubit gates (like CNOT or $R_{zz}$) between physically adjacent qubits.

**Native Gate Set:** The hardware only natively supports a specific set of basis gates (e.g., $RZ$, $SX$, $X$, $ECR$). Your abstract gates must be decomposed into these.Noise: Real gates have errors.

### **The Qiskit Primitives: Estimator & Sampler**
Once your circuit is transpiled, you need to run it. In the modern Qiskit ecosystem (Qiskit Runtime), there are two distinct ways to run a job, called Primitives. They serve different purposes in the variational loop.
It is crucial to distinguish between the two main Qiskit Primitives:

1. **The `Estimator`:** Used *during* the hybrid loop. Its job is to return the single expectation value (the cost) $C = \langle H_P \rangle$. The optimizer uses this number to decide on the next $\vec{\theta}$.

* **How it works:** 
  1. We give the `Estimator` our **Ansatz** circuit. 
  2. We give it our **Problem Hamiltonian** ($H_P$). 
  3. It efficiently runs the circuit and performs the necessary measurements *at the hardware level* to return the single number, $C$.

2. **The `Sampler`:** Used *after* the hybrid loop is finished. Once the optimizer has found the best parameters, $\vec{\theta}_{best}$, we run the final circuit $U(\vec{\theta}_{best})$ one last time and use the `Sampler` to get the measurement outcomes (the bitstrings). This shows us the most probable bitstring(s), which are the final *answers* to our optimization problem.

* **Output**: A distribution of bitstrings (e.g., `{'0101': 0.45, '1010': 0.45, ...}`).

This $C$ is the value that gets fed back to the classical optimizer. The entire hybrid loop is a partnership:

* The **Ansatz** (Quantum) prepares the state.

* The **Estimator** (Quantum/Runtime) measures the cost.

* The **Optimizer** (Classical) chooses the next parameters.

* The **Sampler** (Retrieve solution) show the optimal output with highest probability.